# Course Summary

We will cover three fields:
1. Summarize the previous lecture
2. Deploy the models
3. Suggestions about leanrning

## 1. Summarize the previous lecture
1. Preprocessing data

    数据集的标准化，归一化，数据的选择和降维。比如说涉及到距离和梯度的模型，一定得对数据做处理。特征选择基于树的重要性，L1范数
    
2. Classification

    1. 线性模型，lasso, ridge, logistic, linear SVM. 
    2. 如何避免过拟合。用正则化
    3. 非线性。bagging, boosting
    
3. Regression

    大部分模型都是既可作分类也可作回归。重点讲了评估指标。添加了pipline。
4. Model Evaluation

    CV。gridsearch。分类模型的评估指标，准确率不好用，于是用recall，precision，f1，roc来代替
5. Clustering

    k近邻，kmeans，Hierarchical clustering， DBSCAN。涉及到了距离，必须对数据做标准化
    
以上都是基于台大课程讲解的基础上。

## 2. Deploy the models
如何部署模型
1. Linux command
2. Java invoke
3. PMML

### 2.1 如何用linux来调度模型，自动跑

假设我们已经有一组数据，我们想要训练好的数据来预测这个新的数据。首先我们需要把模型保存下来。

In [2]:
# Save the trained model

from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier

X, y = make_blobs(n_samples=10000, n_features=10, centers=100, random_state=0)

clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [3]:
from sklearn.externals import joblib
joblib.dump(clf, "RandomForest.pkl")
# 运行完之后发现当前的目录下多了一个RandomForest.pkl文件，这个就是我们训练好的RF模型
# joblib是用来保存模型到本地的，变为一个二进制的文件

['RandomForest.pkl']

### 那么如何用建好的模型来预测呢？

进入linux终端来进行操作。有一个test.plk的文件，这个就是存有数据集的文件。我们希望每天模型自动来跑这个测试文件。必须得写脚本来实现。写一个test.py来实现功能。

![1](summary_images/1.png)

在vim里直接书写脚本。先导入库，再导入模型。


test.py
```
from sklearn.externals import joblib 
form sklearn.ensemble import RandomForestClassifier
import pickle   # 因为我们刚才保的是pkl格式，所以这里导入这个包来读取文件

with open("test.plk", "rb") as handle:
    testData = pickle.load(handle)
model = joblib.load("RandomForest.pkl")
prediction = model.predict_proba(testData)

print (prediction)
```

有了上面的模型后，回到命令行，用python test.py就能得到预测结果。

![2](summary_images/2.png)
但是我们希望能设置一个固定的时间再跑这个程序，比如早上10：10跑，有具体的时间性需求。


crontab能很好地实现这个功能。Crontab (CRON TABle) is a file which contains the schedule of cron entries to be run and at specified times. [参考文档](http://www.computerhope.com/unix/ucrontab.htm)

你可以理解为有6个参数。

field	allowed values
2. minute	0-59
3. hour	0-23
4. day of month	1-31
5. month	1-12 (or names; see example below)
6. day of week	0-7 (0 or 7 is Sunday, or use names; see below)


例子：`15 6 2 1 * /home/melissa/backup.sh`

含义：Run the shell script /home/melissa/backup.sh on January 2 at 6:15 A.M.


可以在命令行界面修改默认打开的编辑器：`export EDITOR=vim`

`crontab -e` ： 进入vim开始编辑crontab文件。

进入编辑的时候，一定要输入绝对地址，不然找不到对应的程序来运行。比如我们希望找python的绝对地址，找到anaconda里对应的文件。![3](summary_images/3.png). 但我在本地找的时候没有安装python3.5，只是一个环境里有3.5。所以切换环境后找到了地址，然后把这个地址用到crontab文件里。![4](summary_images/4.png)
. 可以用`/Users/xu/anaconda/envs/py35/bin/python test.py`来测试一下能不能正常运行。


crontab文件的格式如下
```
#先写好python路径和要处理的文件路径
* * * * * /Users/xu/anaconda/envs/py35/bin/python /Users/xu/Udacity/sklearn_note/test.py

#然后把* * * * * 的部分设置成具体时间就可以了
#我们希望是每一分钟的话，利用/.
*/1 * * * * /Users/xu/anaconda/envs/py35/bin/python /Users/xu/Udacity/sklearn_note/test.py

#另外如果我们希望把输出的结果保存下来的话，用下面的命令.用一个>好说明是每一次都会把之前的文件覆盖掉，如果用>>的话，每一次的结果都会保存下来，放在文件的最下面
*/1 * * * * /Users/xu/anaconda/envs/py35/bin/python /Users/xu/Udacity/sklearn_note/test.py>temp.txt

#我们希望先打印出时间，再打印出模型的结果
*/1 * * * * data>>temp.txt
```
上面说了那么多，其实真正的crontab文件里，只需要写两行即可
```
*/1 * * * * data>>temp.txt
*/1 * * * * /Users/xu/anaconda/envs/py35/bin/python /Users/xu/Udacity/sklearn_note/test.py>temp.txt
```

过一段时间后，就能查看temp.txt文件了![5](summary_images/5.png)

注意，我是在测试。不用的话必须的吧文件给关了，不然会一直生成文件，知道服务器爆了为止。所以一定要删了文件`crontab -e`进入文件后，把上面的两行删了即可。

这就是整个linux的调度过程

### 2.2 Java invoke

这个可以用java来调用python的文件。

### 2.3 PMML

比如只会python和R的人，在工程上不可能只用这两个，比如用java之类的。那么怎么来调用呢？用PMML。可以把模型变为pmml格式。这个格式就是一个xml文件，可以被任何语言解析。比如python和java都可以调用这一种格式，那么这两种文件就是可以被打通的。这个作者的项目不错，已经测试过了。[here](https://github.com/jpmml/sklearn2pmml)

就是说，我用python建好了模型，然后其他同事用java去调取。

## 3. Suggestions about leanrning

学习建议：

偏理论：中文的话只有西瓜书和统计学习了。

实践：kaggle。真实的数据集才能对模型有更好的理解。在项目中才能认识到特征工程的重要性。多看别人分享的做法

加深理论的学习，python和sklearn都是工具。